In [8]:
import torch
from torchvision.models import vgg19
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import numpy as np
import torch
import os
from PIL import Image, ImageOps, ImageFilter
import random
import time
import pandas as pd
from tqdm import tqdm

from ImageData import getImagesDataloaders


In [2]:
dataDir = "./Artifact/"
device = "cpu"
tsfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [3]:
trainDataloader, validDataloader, testDataloader = getImagesDataloaders(dataDir, 4, tsfms)

In [4]:
class vggClassifier(torch.nn.Module):
    def __init__(self, numClasses):
        super(vggClassifier, self).__init__()
        vgg = vgg19(weights = "DEFAULT")
        self.featureExtractor = vgg.features
        self.avgpool = torch.nn.AdaptiveAvgPool2d((7, 7))
        print(list(vgg.classifier.children()))
        self.class1, _, _, self.class2, _, _, _  = list(vgg.classifier.children())
        self.class3 = torch.nn.Linear(in_features = 4096, out_features = numClasses)
        # self.classifier = torch.nn.Sequential(
        #     classifierLayer[0],
        #     torch.nn.ReLU(inplace=True),
        #     classifierLayer[1],
        #     torch.nn.ReLU(inplace=True),
        #     torch.nn.Linear(in_features = 1024, out_features = numClasses)
        # )

    def forward(self, x):
        #do something 
        x = self.featureExtractor(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        # x = self.classifier(x)
        x = self.class1(x)
        x = self.class2(x)
        x = self.class3(x)

        return x





In [6]:
model = vggClassifier(2).to(device)

[Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=1000, bias=True)]


True

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-5)

mse = 0

for data in tqdm(trainDataloader, "training"):
    img = data[0]
    label = data[1]

    img, label = img.to(device), label.to(device)
    
    pred = model(img)
    loss = criterion(pred, label)
    mse += loss.item()

    loss.backward()
    optim.step()

print(mse)

    

    

training:   0%|          | 1/464685 [00:02<361:53:52,  2.80s/it]


IndexError: Target 3 is out of bounds.